用三层神经网络实现手写数字识别
输入是784
      W1=[784,256]
layer1  256
      W2=[256,256]
layer2  256
      W3=[256,10]
输出是10


1，数据准备

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

print(mnist.train.images.shape,mnist.train.labels.shape)
print(mnist.test.images.shape,mnist.test.labels.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)


2,准备好placeholder

In [2]:
batch_size = 126
x = tf.placeholder(tf.float32,[batch_size,784],name="x")
y_ = tf.placeholder(tf.float32,[batch_size,10],name="y")

3,初始化参数/权重

In [3]:
hidden1 = 256

hidden2 = 256

classes = 10


keep_prob = tf.placeholder(tf.float32)

4,拿到每个类别的score

In [4]:
with tf.variable_scope("fc_1"):
    w1 = tf.Variable(tf.random_normal([784,hidden1]),name="w1")
    b1 = tf.Variable(tf.zeros([hidden1]),name="b1")
    logits = tf.matmul(x,w1) + b1
    logits = tf.nn.relu(logits,name="relu")
    
with tf.variable_scope("dropout"):
    w2 = tf.Variable(tf.random_normal([hidden1,hidden2]),name="w2")
    b2 = tf.Variable(tf.zeros([hidden2]),name="b2")
    logits = tf.nn.dropout(logits,keep_prob)

with tf.variable_scope("fc_2"):
    w3 = tf.Variable(tf.random_normal([hidden2,classes]),name="w3")
    b3 = tf.Variable(tf.zeros([classes]),name="b3")
    logits = tf.matmul(logits,w2) + b2
    logits = tf.nn.relu(logits,name="relu")
    
with tf.variable_scope("output"):
    logits = tf.matmul(logits,w3) + b3


5,计算多分类的softmax的loss function

In [5]:
#交叉熵损失
entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y_,name="loss")
#求平均值
loss = tf.reduce_mean(entropy)
loss_summary = tf.summary.scalar("my_loss",loss)

6,准备好optimizer

In [8]:
learning_rate = 0.01
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

7,在session里执行graph里定义的运算

In [9]:
n_epoch = 30
init = tf.global_variables_initializer()
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs/logistic_mul",sess.graph)
    sess.run(init)
    n_batch = int(mnist.train.num_examples/batch_size)
    
    merged_summary = tf.summary.merge_all()
    
    for i in range(n_epoch):
        total_loss = 0
        for j in range(n_batch):
            x_batch,y_batch = mnist.train.next_batch(batch_size)
            _,loss_batch,summary_loss = sess.run([optimizer,loss,merged_summary],feed_dict={x:x_batch,y_:y_batch,keep_prob:0.75})
            total_loss += loss_batch
            writer.add_summary(summary_loss)
        print("epoch {0}:{1}".format(i,total_loss))
    print("optimizer finished")
    
    #测试模型
    preds = tf.nn.softmax(logits)
    correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(y_,1))
    accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32))
    
    n_batch = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    
    for i in range(n_batch):
        x_batch,y_batch = mnist.train.next_batch(batch_size)
        accuracy_batch = sess.run([accuracy],feed_dict={x:x_batch,y_:y_batch,keep_prob:1.0})
        total_correct_preds += accuracy_batch[0]
    print("accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    
    writer.close()
        
    

epoch 0:32263.974585056305
epoch 1:5613.664098978043
epoch 2:2449.860861301422
epoch 3:1224.066454231739
epoch 4:748.3581049144268
epoch 5:600.3050388395786
epoch 6:500.15025421977043
epoch 7:455.4277083873749
epoch 8:439.87804558873177
epoch 9:407.3439503312111
epoch 10:388.59208285808563
epoch 11:406.6936429440975
epoch 12:396.4006362259388
epoch 13:384.2311804294586
epoch 14:390.69986352324486
epoch 15:370.33665123581886
epoch 16:346.89420452713966
epoch 17:345.013072937727
epoch 18:338.0301823914051
epoch 19:317.61170426011086
epoch 20:322.8593774139881
epoch 21:321.27311313152313
epoch 22:293.5258673131466
epoch 23:269.31469586491585
epoch 24:261.0479240119457
epoch 25:277.50244414806366
epoch 26:249.17410688102245
epoch 27:251.14440834522247
epoch 28:224.36869294941425
epoch 29:212.07960914075375
optimizer finished
accuracy 0.9102
